In [1]:
from pyspark.mllib.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import  ParamGridBuilder
from pyspark.ml.tuning import  CrossValidator
from pyspark.ml import Pipeline
from pyspark.ml import PipelineModel

In [2]:
# input and output folders
labeledData = "ex_dataValidation/labeledData.csv"
unlabeledData = "ex_dataValidation/unlabeledData.csv"
outputPath = "predictionsLRPipelineTuning/"

In [3]:
# Create a DataFrame from labeledData.csv
# Training data in raw format
labeledDataDF = spark.read.load(labeledData,\
                     format="csv",\
                     header=True,\
                     inferSchema=True)

In [4]:
#labeledDataDF.show()

In [5]:
# *************************
# Training step
# *************************

In [6]:
# Define an assembler to create a column (features) of type Vector 
# containing the double values associated with columns attr1, attr2, attr3
assembler = VectorAssembler(inputCols=["attr1", "attr2", "attr3"],\
                            outputCol="features")

In [7]:
# Create a LogisticRegression object.  
# LogisticRegression is an Estimator that is used to 
# create a classification model based on logistic regression.
lr = LogisticRegression()

In [8]:
# We can set the values of the parameters of the 
# Logistic Regression algorithm using the setter methods.
# There is one set method for each parameter
# For example, we are setting the number of maximum iterations to 10
# and the regularization parameter. to 0.0.1
lr.setMaxIter(10)
lr.setRegParam(0.01)

LogisticRegression_4065b67550e5

In [9]:
# Define a pipeline that is used to create the logistic regression
# model on the training data. The pipeline includes also 
# the preprocessing step
pipeline = Pipeline().setStages([assembler, lr])

In [10]:
# We use a ParamGridBuilder to construct a grid of parameter values to 
# search over.
# We set 3 values for lr.setMaxIter and 2 values for lr.regParam.
# This grid will evaluate 3 x 2 = 6 parameter settings for 
# the input pipeline.
paramGrid = ParamGridBuilder()\
.addGrid(lr.maxIter, [10, 100, 1000])\
.addGrid(lr.regParam, [0.1, 0.01])\
.build()

In [11]:
# We now treat the Pipeline as an Estimator, wrapping it in a 
#  CrossValidator instance. This allows us to jointly choose parameters 
# for all Pipeline stages.
# CrossValidator requires 
# - an Estimator
# - a set of Estimator ParamMaps
# - an Evaluator.
cv = CrossValidator()\
.setEstimator(pipeline)\
.setEstimatorParamMaps(paramGrid)\
.setEvaluator(BinaryClassificationEvaluator())\
.setNumFolds(3)

In [12]:
# Run cross-validation. The result is the logistic regression model 
# based on the best set of parameters (based on the results of the 
# cross-validation operation).
tunedLRmodel = cv.fit(labeledDataDF)

In [13]:
# Now, the tuned classification model can be used to predict the class label
# of new unlabeled data

In [14]:
# Create a DataFrame from unlabeledData.csv
# Unlabeled data in raw format
unlabeledData = spark.read.load(unlabeledData,\
                     format="csv", header=True, inferSchema=True)

In [15]:
#unlabeledData.printSchema()
#unlabeledData.show()

In [16]:
# Make predictions on the unlabled data using the transform() method of the 
# trained tuned classification model transform uses only the content of 'features' 
# to perform the predictions. The model is associated with the pipeline and hence
# also the assembler is executed
predictionsDF = tunedLRmodel.transform(unlabeledData)

In [17]:
# The returned DataFrame has the following schema (attributes)
# - features: vector (values of the attributes)
# - label: double (value of the class label)
# - rawPrediction: vector (nullable = true)
# - probability: vector (The i-th cell contains the probability that the current 
#	record belongs to the i-th class
# - prediction: double (the predicted class label)

# Select only the original features (i.e., the value of the original attributes 
# attr1, attr2, attr3) and the predicted class for each record
predictions = predictionsDF.select("attr1", "attr2", "attr3", "prediction")

In [18]:
#predictions.printSchema()
#predictions.show()

In [19]:
# Save the result in an HDFS output folder
predictions.write.csv(outputPath, header="true")